In [14]:
import math
import numpy as np
from graphviz import Digraph
import pydot

In [ ]:
input = [1, 2, 3, 4]
[0 for _ in range(len(input))]

[0, 0, 0, 0]

In [53]:
# import numpy as np

# class Params:
#     """Class representing parameters to a mini Neural Net aka WizNet."""
    
#     activationFunc = {
#         "Sigmoid": lambda x: 1 / (1 + np.exp(-x)),
#         "ReLU": lambda x: np.maximum(0, x),
#         "Tanh": lambda x: np.tanh(x),
#     }
    
#     def __init__(self, value, grad=0, dtype=None):
#         """
#         Initializes a parameter.

#         Args:
#             value (float or list/np.ndarray): The value of the parameter.
#             grad (float or list/np.ndarray, optional): The gradient of the parameter. Default is 0.
#             dtype (str, optional): The type of the parameter. Default is None.
#         """
#         self.value = value
#         self.grad = grad
#         self.type = dtype
    
#     def __add__(self, biasParam):
#         """
#         Adds two parameters.

#         Args:
#             biasParam (Params): The bias parameter to add.

#         Returns:
#             Params: A new Params object with the summed value.
#         """
#         total = self.value + biasParam.value
#         return Params(total)
    
#     def __mul__(self, dataParam):
#         """
#         Multiplies the parameter value with another parameter using dot product.

#         Args:
#             dataParam (Params): The data parameter to multiply with.

#         Returns:
#             Params: A new Params object with the resulting value.
#         """
#         total = np.dot(self.value, dataParam.value)
#         return Params(total)
    
#     def __repr__(self):
#         """
#         Returns a string representation of the parameter.

#         Returns:
#             str: A string describing the parameter type, value, and gradient.
#         """
#         return f"Parameter type: {self.type}, Value: {self.value}, Grad: {self.grad}"

#     def weighted_sum(self, param1, param2):
#         """
#         Calculates the weighted sum of weights, data, and bias parameters.

#         Args:
#             param1 (Params): The first parameter (e.g., weights).
#             param2 (Params): The second parameter (e.g., data).

#         Returns:
#             Params: A new Params object with the weighted sum.
#         """
#         variables = {
#             self.type: self,
#             param1.type: param1,
#             param2.type: param2
#         }
        
#         weighted_sum = np.dot(variables['weight'].value, variables['data'].value) + variables['bias'].value
        
#         return Params(weighted_sum, dtype='data')
        

# class Weights(Params):
#     """Class representing weights input to a neuron / nn."""
    
#     def __init__(self, value):
#         """
#         Initializes weights parameters to specified value.

#         Args:
#             value (float or list/np.ndarray): The value of the weights.
#         """
#         super().__init__(value)
#         self.type = "weight"

# class Bias(Params):
#     """Class representing bias input to a neuron / nn."""
    
#     def __init__(self, value):
#         """
#         Initializes bias parameter to specified value.

#         Args:
#             value (float or list/np.ndarray): The value of the bias.
#         """
#         super().__init__(value)
#         self.type = "bias"

# class Data(Params):
#     """Class representing data input to a neuron / nn."""
    
#     def __init__(self, value):
#         """
#         Initializes data parameter to specified value.

#         Args:
#             value (float or list/np.ndarray): The initial value of the data.
#         """
#         super().__init__(value)  
#         self.type = "data"


In [1]:
import numpy as np
import graphviz
from graphviz import Digraph

class Params:
    """Class representing parameters to a mini Neural Net aka WizNet."""
    
    activationFunc = {
        "Sigmoid": lambda x: 1 / (1 + np.exp(-x)),
        "ReLU": lambda x: np.maximum(0, x),
        "Tanh": lambda x: np.tanh(x),
    }
    
    def __init__(self, value, grad=0, dtype=None):
        """
        Initializes a parameter.

        Args:
            value (float or list/np.ndarray): The value of the parameter.
            grad (float or list/np.ndarray, optional): The gradient of the parameter. Default is 0.
            dtype (str, optional): The type of the parameter. 
        """
        self.value = value
        self.grad = grad
        self.type = dtype
        self._prev = set()
        self._op = None

    def __add__(self, other):
        """
        Adds two parameters.

        Args:
            other (Params): The other parameter to add.

        Returns:
            Params (Data): New Params object with the summed value.
        """
        result = Params(self.value + other.value, dtype='data')
        result._prev = {self, other}
        result._op = '+'
        return result
    
    def __mul__(self, other):
        """
        Multiplies the parameter value with another parameter using dot product.

        Args:
            other (Params): The other parameter to multiply with.

        Returns:
            Params: New Params object with the resulting value.
        """
        result = Params(np.dot(self.value, other.value), dtype='product')
        result._prev = {self, other}
        result._op = '*'
        return result
    
    def __repr__(self):
        """
        Returns a string representation of the parameter.

        Returns:
            str: String describing the parameter type, value, and gradient.
        """
        return f"Parameter type: {self.type}, Value: {self.value}, Grad: {self.grad}"

    def weighted_sum(self, weight, data, bias):
        """
        Calculates the weighted sum of weights, data, and bias parameters.

        Args:
            weight (Params): The weights parameter.
            data (Params): The data parameter.
            bias (Params): The bias parameter.

        Returns:
            Params: New Params object with the weighted sum.
        """
        result = Params(Params.activationFunc[function](self.value), dtype = "Activation")
        result._prev = {self, bias}
        result._op = function
        return result
        
        return weight * data + bias
    
    def apply_activation(self, function='Sigmoid'):
        result = Params(Params.activationFunc[function](self.value), dtype = "Activation")
        result._prev = {self}
        result._op = function
        return result
    
    
    def backprop(self):
        if self._prev:
            if self._op == "+":
                for param in self._prev:
                    param.grad = 1 * self.grad  # Accumulate gradients
            elif self._op == "*":
                product = 1
                for param in self._prev:
                    product *= param.value
                for param in self._prev:
                    param.grad = self.grad * (product / param.value)  # Accumulate gradients
            elif self._op in Params.activationFunc.keys():
                if self._op == "Sigmoid":
                    grad = self.value * (1 - self.value)  # Sigmoid derivative
                elif self._op == "Tanh":
                    grad = 1 - self.value ** 2  # Tanh derivative
                elif self._op == "ReLU":
                    grad = 1 if self.value > 0 else 0  # ReLU derivative
                for param in self._prev:
                    param.grad = self.grad * grad  # Apply chain rule
                    # param.grad = self.grad * grad  # Apply chain rule

            for param in self._prev:
                param.backprop()

                    
            
            


class Weights(Params):
    """Class representing weights input to a neuron / nn."""
    
    def __init__(self, value):
        """
        Initializes weights parameters to specified value.

        Args:
            value (float or list/np.ndarray): The value of the weights.
        """
        super().__init__(value)
        self.type = "weight"

class Bias(Params):
    """Class representing bias input to a neuron / nn."""
    
    def __init__(self, value):
        """
        Initializes bias parameter to specified value.

        Args:
            value (float or list/np.ndarray): The value of the bias.
        """
        super().__init__(value)
        self.type = "bias"

class Data(Params):
    """Class representing data input to a neuron / nn."""
    
    def __init__(self, value):
        """
        Initializes data parameter to specified value.

        Args:
            value (float or list/np.ndarray): The initial value of the data.
        """
        super().__init__(value)  
        self.type = "data"


# Visualization code form Andrej Karpathy Micrograd
# Reference: https://github.com/karpathy/micrograd

def trace(root):
    nodes, edges = set(), set()
    
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
                
    build(root)
    return nodes, edges

def format_value(value):
    if isinstance(value, np.ndarray):
        return np.array2string(value, precision=4, separator=',')
    else:
        return f"{value:.4f}"

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # LR = left to right
    
    nodes, edges = trace(root)
    # print(nodes, edges)
    for n in nodes:
        uid = str(id(n))
        dot.node(name=uid, label="{ %s | value %s | grad %.4f }" % (n.type, format_value(n.value), n.grad), shape='record')
        if n._op:
            dot.node(name=uid + n._op, label=n._op)
            # print(uid, n._op)
            dot.edge(uid + n._op, uid)

    # print(edges)
    for n1, n2 in edges:
        print(n2, n2._op)
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot




In [4]:
# a = Data(22)
# b = Weights(.5)
# c = Bias(23)
# d = a*b + c
# e = d.apply_activation(function="ReLU")


# e.grad = 2
# e.backprop()
# draw_dot(e)

